In [130]:
import pandas as pd
import numpy as np

In [135]:
df = pd.read_csv('train.csv')

In [136]:
df.head()

,user_id,order_completed_at,cart
0,2,2015-03-22 09:25:46,399
1,2,2015-03-22 09:25:46,14
2,2,2015-03-22 09:25:46,198
3,2,2015-03-22 09:25:46,88
4,2,2015-03-22 09:25:46,157


In [137]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3123064 entries, 0 to 3123063
Data columns (total 3 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   user_id             int64 
 1   order_completed_at  object
 2   cart                int64 
dtypes: int64(2), object(1)
memory usage: 71.5+ MB


In [138]:
df['user_id'].nunique()

20000

In [151]:
df.order_completed_at = pd.to_datetime(df.order_completed_at)
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3123064 entries, 0 to 3123063
Data columns (total 3 columns):
 #   Column              Dtype         
---  ------              -----         
 0   user_id             int64         
 1   order_completed_at  datetime64[ns]
 2   cart                int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 71.5 MB
None


,user_id,order_completed_at,cart
0,2,2015-03-22 09:25:46,399
1,2,2015-03-22 09:25:46,14
2,2,2015-03-22 09:25:46,198
3,2,2015-03-22 09:25:46,88
4,2,2015-03-22 09:25:46,157


In [147]:
sample_df = pd.read_csv('sample_submission.csv')

In [148]:
sample_df['user'] = sample_df['id'].apply(lambda x: x.split(';')).apply(lambda x: int(x[0]))
sample_df['category'] = sample_df['id'].apply(lambda x: x.split(';')).apply(lambda x: int(x[1]))
sample_df.head()

In [152]:
uniq_users = pd.unique(sample_df['user'])
uniq_categories = pd.unique(sample_df['category'])

In [155]:
df = df.loc[df['user_id'].isin(uniq_users) & df['cart'].isin(uniq_categories)]

print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 2395657 entries, 16 to 3123063
Data columns (total 3 columns):
 #   Column              Dtype         
---  ------              -----         
 0   user_id             int64         
 1   order_completed_at  datetime64[ns]
 2   cart                int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 73.1 MB
None


,user_id,order_completed_at,cart
16,3,2015-06-18 16:15:33,399
17,3,2015-07-04 14:05:22,399
18,4,2015-07-08 06:59:04,54
19,4,2015-07-08 06:59:04,55
20,5,2015-07-22 08:16:24,409


In [156]:
orders = df.groupby(['order_completed_at', 'user_id']).apply(lambda x: list(x['cart'])).reset_index()
orders = orders.rename(columns = {0: 'carts_in_order'})
orders['len_cart'] = orders.carts_in_order.apply(lambda x: len(x))

C:\Users\Борис\AppData\Local\Temp\ipykernel_9008\3801794931.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  orders = df.groupby(['order_completed_at', 'user_id']).apply(lambda x: list(x['cart'])).reset_index()


In [157]:
orders.head()

,order_completed_at,user_id,carts_in_order,len_cart
0,2015-06-18 16:15:33,3,[399],1
1,2015-07-04 14:05:22,3,[399],1
2,2015-07-08 06:59:04,4,"[54, 55]",2
3,2015-07-22 08:16:24,5,"[409, 88, 376, 14, 423, 23, 92, 29, 386, 440, ...",12
4,2015-08-12 10:33:44,3,"[804, 134]",2


In [158]:
orders.user_id.value_counts()

user_id
75       187
5992     165
458      163
1156     155
1431     154
        ... 
6057       3
17236      3
17248      3
17250      3
16887      3
Name: count, Length: 13036, dtype: int64

In [161]:
def count_carts_number(x):
    count = 0
    for order in x['carts_in_order']:
        count += len(order)
    return count
users_carts_mean_number = orders.groupby('user_id').apply(count_carts_number)/orders.groupby('user_id')['carts_in_order'].count()
users_carts_mean_number = users_carts_mean_number.reset_index()
users_carts_mean_number.rename(columns = {0:'mean_carts'}, inplace = True)
users_carts_mean_number

C:\Users\Борис\AppData\Local\Temp\ipykernel_9008\1170639208.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  users_carts_mean_number = orders.groupby('user_id').apply(count_carts_number)/orders.groupby('user_id')['carts_in_order'].count()


,user_id,mean_carts
0,0,14.666667
1,1,4.111111
2,3,8.857143
3,4,14.000000
4,5,7.733333
...,...,...
13031,19994,6.800000
13032,19995,3.000000
13033,19996,5.666667
13034,19997,14.000000


In [162]:
from itertools import groupby
import collections

def list_of_favourite_carts(x): # создает отсортированный (по частоте) список заказанных категорий пользователя
    carts_list = []
    for order in x['carts_in_order']:
        carts_list.extend(order)
    counts = collections.Counter(carts_list)
    new_list = sorted(carts_list, key=lambda y: (counts[y], y), reverse=True)

    cart_list = [el for el, _ in groupby(new_list)]

    return cart_list
users_favourite_carts = orders.groupby('user_id').apply(list_of_favourite_carts).reset_index()
users_favourite_carts

C:\Users\Борис\AppData\Local\Temp\ipykernel_9008\2440066861.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  users_favourite_carts = orders.groupby('user_id').apply(list_of_favourite_carts).reset_index()


,user_id,0
0,0,"[57, 441, 405, 379, 84, 82, 14, 821, 803, 799,..."
1,1,"[55, 798, 169, 88, 14, 812, 803, 421, 307, 302..."
2,3,"[399, 383, 382, 57, 43, 41, 19, 430, 398, 179,..."
3,4,"[398, 57, 420, 98, 54, 712, 102, 100, 84, 61, ..."
4,5,"[712, 23, 92, 440, 398, 88, 84, 57, 813, 798, ..."
...,...,...
13031,19994,"[393, 67, 19, 17, 14, 820, 443, 433, 430, 427,..."
13032,19995,"[712, 398, 393, 84, 67, 57, 31, 14, 0]"
13033,19996,"[812, 804, 443, 431, 409, 404, 396, 393, 231, ..."
13034,19997,"[61, 57, 55, 49, 479, 443, 430, 420, 398, 383,..."


In [163]:
users_data = users_carts_mean_number.merge(users_favourite_carts, on = 'user_id')
users_data.rename(columns = {0: 'favourite_carts'}, inplace = True)
users_data

,user_id,mean_carts,favourite_carts
0,0,14.666667,"[57, 441, 405, 379, 84, 82, 14, 821, 803, 799,..."
1,1,4.111111,"[55, 798, 169, 88, 14, 812, 803, 421, 307, 302..."
2,3,8.857143,"[399, 383, 382, 57, 43, 41, 19, 430, 398, 179,..."
3,4,14.000000,"[398, 57, 420, 98, 54, 712, 102, 100, 84, 61, ..."
4,5,7.733333,"[712, 23, 92, 440, 398, 88, 84, 57, 813, 798, ..."
...,...,...,...
13031,19994,6.800000,"[393, 67, 19, 17, 14, 820, 443, 433, 430, 427,..."
13032,19995,3.000000,"[712, 398, 393, 84, 67, 57, 31, 14, 0]"
13033,19996,5.666667,"[812, 804, 443, 431, 409, 404, 396, 393, 231, ..."
13034,19997,14.000000,"[61, 57, 55, 49, 479, 443, 430, 420, 398, 383,..."


In [166]:
users_data.mean_carts = users_data.mean_carts.apply(lambda x: round(x))
users_data = users_data.astype({'mean_carts': int})
users_data

,user_id,mean_carts,favourite_carts
0,0,15,"[57, 441, 405, 379, 84, 82, 14, 821, 803, 799,..."
1,1,4,"[55, 798, 169, 88, 14, 812, 803, 421, 307, 302..."
2,3,9,"[399, 383, 382, 57, 43, 41, 19, 430, 398, 179,..."
3,4,14,"[398, 57, 420, 98, 54, 712, 102, 100, 84, 61, ..."
4,5,8,"[712, 23, 92, 440, 398, 88, 84, 57, 813, 798, ..."
...,...,...,...
13031,19994,7,"[393, 67, 19, 17, 14, 820, 443, 433, 430, 427,..."
13032,19995,3,"[712, 398, 393, 84, 67, 57, 31, 14, 0]"
13033,19996,6,"[812, 804, 443, 431, 409, 404, 396, 393, 231, ..."
13034,19997,14,"[61, 57, 55, 49, 479, 443, 430, 420, 398, 383,..."


In [168]:
from tqdm import tqdm
d = {}
indexer = 0

for index, row in tqdm(users_data.iterrows()):
    mean_carts = row.mean_carts
    user_id = row.user_id
    favourite_carts = row.favourite_carts

    for i in range(len(favourite_carts)):
        id_to_add = str(user_id) + ';' + str(favourite_carts[i])
        if i < mean_carts:
            target_to_add = 1
        else:
            target_to_add = 0
        d[indexer] = {'id':id_to_add, 'target': target_to_add}
        indexer +=1

submission = pd.DataFrame.from_dict(d, 'index')

13036it [00:00, 13774.39it/s]


In [169]:
submission.astype({'target':int})

,id,target
0,0;57,1
1,0;441,1
2,0;405,1
3,0;379,1
4,0;84,1
...,...,...
790444,19998;61,0
790445,19998;31,0
790446,19998;29,0
790447,19998;26,0


In [170]:
sample = pd.read_csv('sample_submission.csv')

In [171]:
sample.drop('target', axis = 1, inplace = True)
submission = submission.merge(sample, how = 'right')

In [187]:
submission.to_csv('sub2.csv', index=False)
# sub1 == 0.46475
# sub2 == 0.46477

In [172]:
submission

,id,target
0,0;133,0
1,0;5,0
2,0;10,0
3,0;396,0
4,0;14,1
...,...,...
790444,19998;26,0
790445,19998;31,0
790446,19998;29,0
790447,19998;798,0


In [182]:
qweqwe = pd.read_csv('sub1.csv')
mask = qweqwe==submission
print(sum(submission.target))
print(sum(qweqwe.target))

SyntaxError: invalid syntax (1121086979.py, line 2)

In [183]:
first = submission.target
second = qweqwe.target

In [184]:
first == second

0         True
1         True
2         True
3         True
4         True
          ... 
790444    True
790445    True
790446    True
790447    True
790448    True
Name: target, Length: 790449, dtype: bool

In [185]:
fir_df = pd.read_csv('sub2.csv')
first = fir_df.target
second = qweqwe.target

In [186]:
first == second

0         True
1         True
2         True
3         True
4         True
          ... 
790444    True
790445    True
790446    True
790447    True
790448    True
Name: target, Length: 790449, dtype: bool